In [ ]:
from collections import Counter
vocab=Counter()
import re

In [ ]:
f = open('./train.txt', 'r')

sentences = []
sentence = []
ner_set = set()
  
for line in f:
    if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence=[]
        continue
    splits = line.split(' ')
    # 공백을 기준으로 속성을 구분한다.
    splits[-1] = re.sub(r'\n', '', splits[-1])
    # 개체명 태깅 뒤에 붙어있는 줄바꿈 표시 \n을 제거한다.
    word=splits[0].lower()
    # 단어들은 소문자로 바꿔서 저장한다. 단어의 수를 줄이기 위해서이다.
    vocab[word]=vocab[word]+1
    # 단어마다 빈도 수가 몇 인지 기록한다.
    sentence.append([word, splits[-1]])
    # 단어와 개체명 태깅만 기록한다.
    ner_set.add(splits[-1])
    # set에다가 개체명 태깅을 집어 넣는다. 중복은 허용되지 않으므로
    # 나중에 개체명 태깅이 어떤 종류가 있는지 확인할 수 있다.

In [ ]:
sentences[:3]

In [ ]:
vocab

In [ ]:
len(vocab)

In [ ]:
print(ner_set)

In [ ]:
vocab_sorted=sorted(vocab.items(), key=lambda x:x[1], reverse=True)
# vocab을 빈도수 순으로 정렬한다.
vocab_sorted
# 출력

In [ ]:
word_to_index = {w: i + 2 for i, (w, n) in enumerate(vocab_sorted) if n > 5}
word_to_index['PAD'] = 0  # 패딩을 위해 인덱스 0 할당
word_to_index['OOV'] = 1  # 모르는 단어을 위해 인덱스 1 할당
word_to_index # 출력

In [ ]:
print(len(word_to_index))

In [ ]:
word_to_index['the']

In [ ]:
ner_to_index={}
ner_to_index['PAD'] = 0
i=1
for ner in ner_set:
    ner_to_index[ner]=i
    i=i+1
print(ner_to_index)

In [ ]:
ner_to_index['I-PER']

In [ ]:
data_X = []

for s in sentences:
    temp_X = []
    for w, label in s:
        try:
            temp_X.append(word_to_index.get(w,1))
        except KeyError:
            temp_X.append(word_to_index['OOV'])

    data_X.append(temp_X)
print(data_X)

In [ ]:
index_to_word={}
for key, value in word_to_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key


temp = []
for index in data_X[0] : # 첫번째 샘플 안의 인덱스들에 대해서
    temp.append(index_to_word[index]) # 다시 단어로 변환

print(sentences[0])    
print(temp)

In [ ]:
data_y = []

for s in sentences:
    temp_y = []
    for w, label in s:
            temp_y.append(ner_to_index.get(label))

    data_y.append(temp_y)

In [ ]:
print(data_X[:4]) # X 데이터 4개만 출력
print(data_y[:4]) # y 데이터 4개만 출력

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist([len(s) for s in data_X], bins=50)
plt.xlabel('length of Data')
plt.ylabel('number of Data')
plt.show()

In [ ]:
print(max(len(l) for l in data_X)) # 전체 데이터에서 길이가 가장 긴 샘플의 길이 출력
print(max(len(l) for l in data_y)) # 전체 데이터에서 길이가 가장 긴 샘플의 길이 출력

In [ ]:
max_len=70
from keras.preprocessing.sequence import pad_sequences
pad_X = pad_sequences(data_X, padding='post', maxlen=max_len)
# data_X의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자 0으로 채움.
pad_y = pad_sequences(data_y, padding='post', maxlen=max_len)
# data_y의 모든 샘플들의 길이를 맞출 때 뒤의 공간에 숫자0으로 채움.

In [ ]:
print(min(len(l) for l in pad_X)) # 모든 데이터에서 길이가 가장 짧은 샘플의 길이 출력
print(min(len(l) for l in pad_y)) # 모든 데이터에서 길이가 가장 짧은 샘플의 길이 출력

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad_X, pad_y, test_size=.2, random_state=777)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

In [ ]:
n_words = len(word_to_index)
n_labels = len(ner_to_index)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=n_words, output_dim=16, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(TimeDistributed(Dense(n_labels, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
from keras.utils import np_utils
y_train2 = np_utils.to_categorical(y_train)
y_train2[0][0]

In [ ]:
model.fit(X_train, y_train2, epochs=8)

In [ ]:
y_test2 = np_utils.to_categorical(y_test)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test2)[1]))

In [ ]:
import numpy as np

index_to_word={}
for key, value in word_to_index.items():
    index_to_word[value] = key

index_to_ner={}
for key, value in ner_to_index.items():
    index_to_ner[value] = key


i=10 # 확인하고 싶은 테스트용 샘플의 인덱스.
y_predicted = model.predict(np.array([X_test[i]])) # 입력한 테스트용 샘플에 대해서 예측 y를 리턴
y_predicted = np.argmax(y_predicted, axis=-1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.
true = np.argmax(y_test2[i], -1) # 원-핫 인코딩을 다시 정수 인코딩으로 변경함.

print("{:15}|{:5}|{}".format("단어", "실제값", "예측값"))
print(35 * "-")

for w, t, pred in zip(X_test[i], true, y_predicted[0]):
    if w != 0: # PAD값은 제외함.
        print("{:17}: {:7} {}".format(index_to_word[w], index_to_ner[t], index_to_ner[pred]))